# Reading from MySQL

As part of Week 2 of JHU's Learning Data Science course.
___

Useful high-level links:

* [Wikipedia](http://en.wikipedia.org/wiki/MySQL)
* [The MySQL website](http://www.mysql.com/)
* [RMySQL Vignette](http://cran.r-project.org/web/packages/RMySQL/RMySQL.pdf)
* [MySQL and R](http://www.r-bloggers.com/mysql-and-r/)

An alternative to the package used for database access below is RODBC. This provides an interface to all kinds of databases, including PostgreQL, MySQL, MS Access, and SQLite.

**Tutorial**: http://cran.r-project.org/web/packages/RODBC/vignettes/RODBC.pdf

**Help file**: http://cran.r-project.org/web/packages/RODBC/RODBC.pdf

In [1]:
library(RMySQL)

Loading required package: DBI


(Apparently if you're running this on Windows this is modestly painful. [A Windows guide.](http://www.ahschulz.de/2013/07/23/installing-rmysql-under-windows/))

## A reference database
We will pull from and make use of the [UCSC Genome Database.](http://genome.ucsc.edu/goldenPath/help/mysql.html)

From the website:

> You can connect to the US MySQL server using the command:
>
> `mysql --user=genome --host=genome-mysql.soe.ucsc.edu -A -P 3306`


## Connecting to and Listing Databases

We will use the `dbConnect` tool to make a connection to the MySQL server. The `dbConnect` function can be used for a number of DBMS tools - we specify which one with the `drv` argument.

In [3]:
?dbConnect

dbConnect {DBI},R Documentation
drv,"an object that inherits from DBIDriver, or an existing DBIConnection object (in order to clone an existing connection)."
...,"authentication arguments needed by the DBMS instance; these typically include user, password, host, port, dbname, etc. For details see the appropriate DBIDriver."


In [2]:
ucscDB <- dbConnect(MySQL(),
                   user="genome",
                   host="genome-mysql.cse.ucsc.edu")

In [5]:
result <- dbGetQuery(ucscDB,"show databases;"); dbDisconnect(ucscDB);

[1] TRUE

In [6]:
result

Database
information_schema
ailMel1
allMis1
anoCar1
anoCar2
anoGam1
apiMel1
apiMel2
aplCal1
aptMan1


In [8]:
class(result)

[1] "data.frame"

## Connecting to a database and listing tables
Above, we've connected to the server and listed the databases available. Let's hook into one and see what's inside.

In [47]:
hg19 <- dbConnect(MySQL(), user="genome", db="hg19",
                 host="genome-mysql.cse.ucsc.edu")
allTables <- dbListTables(hg19)
length(allTables)

[1] 11107

In [48]:
allTables[1:5]

[1] "HInv"         "HInvGeneMrna" "acembly"      "acemblyClass" "acemblyPep"

## Pulling information from a single table
Having viewed a number of databases, let's choose one and connect to it.

In [50]:
dbListFields(hg19, "affyU133Plus2")

[1] "bin"         "matches"     "misMatches"  "repMatches"  "nCount"     
 [6] "qNumInsert"  "qBaseInsert" "tNumInsert"  "tBaseInsert" "strand"     
[11] "qName"       "qSize"       "qStart"      "qEnd"        "tName"      
[16] "tSize"       "tStart"      "tEnd"        "blockCount"  "blockSizes" 
[21] "qStarts"     "tStarts"

In [51]:
dbGetQuery(hg19, "select count(*) from affyU133Plus2")

count(*)
58463


In [52]:
dbClearResult(query)

[1] TRUE

A general routine for getting data:
1. Connect to the DB you're interested in.
2. Send a DB query to the database - the query is now stored at the database.
3. Ask to fetch the data.
4. When you're done, clear the result.
5. When you're done, disconnect from the DB.

In [61]:
query <- dbSendQuery(hg19, "select * from affyU133Plus2 where misMatches between 1 and 3")
affyU133Plus2Mis <- fetch(query)
affyU133Plus2MisSmall <- fetch(query,n=10)
dbClearResult(query)

Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 0 imported as numeric”Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 1 imported as numeric”Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 2 imported as numeric”Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 3 imported as numeric”Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 4 imported as numeric”Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 5 imported as numeric”Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 6 imported as numeric”Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 7 imported as numeric”Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 8 imported as numeric”Warning message in .local(conn, statement, ...):
“Unsigned INTEGER in col 11 imported as numeric”Warning message in .local(conn, stateme

[1] TRUE

In [64]:
dim(affyU133Plus2Mis)

[1] 500  22

In [65]:
dim(affyU133Plus2MisSmall)

[1] 10 22

Goodbye!

In [66]:
dbDisconnect(hg19)

[1] TRUE